In [1]:
from visual_behavior import database as db

from allensdk.internal.api import PostgresQueryMixin
from allensdk.core.authentication import credential_injector
from allensdk.core.auth_config import LIMS_DB_CREDENTIAL_MAP

In [2]:
def lims_query(query):
    api = (credential_injector(LIMS_DB_CREDENTIAL_MAP)(PostgresQueryMixin)())
    conn = api.get_connection()
    cur = conn.cursor()

    df = pd.read_sql(query, conn)

    conn.close()

    return df

In [3]:
def osid_to_uuid(osid):
    query = 'select foraging_id from ophys_sessions where id = {}'.format(osid)
    return lims_query(query).values[0][0]

In [4]:
def get_stage_from_db(osid):
    uuid = osid_to_uuid(osid)
    summary = db.get_behavior_session_summary()
    res = summary.query('behavior_session_uuid == "{}"'.format(uuid))
    return res['stage'].iloc[0] if len(res) > 0 else None

In [5]:
def get_stage_from_pkl(osid):
    wkf = db.get_well_known_files(osid).set_index('name')
    data = pd.read_pickle(''.join([wkf.loc['StimulusPickle']['storage_directory'],wkf.loc['StimulusPickle']['filename']]))
    return data['params']['stage']

In [6]:
ophys_session_ids = [
    908810528,
    875293691,
    873248441,
    868621477,
    867649640,
    929253634,
    927279477,
    894836178,
    893993015,
    893234928,
    888945661,
    886154222,
    884353347,
    884545175,
    871524137,
    868615577,
    867016776,
    893940179,
    893238904,
    859459978,
    858899525,
    857759922,
    857075165,
    856337979,
    855898523,
    852023722,
    847488953,
    841817171
]

In [7]:
stage_dict = {}

summary = db.get_behavior_session_summary()
for osid in ophys_session_ids:
    uuid= osid_to_uuid(osid)
    
    res = summary.query('behavior_session_uuid == "{}"'.format(uuid))
    stage_dict.update({
        osid: get_stage_from_db(osid) if get_stage_from_db(osid) is not None else get_stage_from_pkl(osid)
    })

In [8]:
stage_dict

{908810528: 'OPHYS_7_receptive_field_mapping',
 875293691: 'OPHYS_1_images_A',
 873248441: 'OPHYS_1_images_A',
 868621477: 'OPHYS_2_images_A_passive',
 867649640: 'OPHYS_1_images_A',
 929253634: 'OPHYS_7_receptive_field_mapping',
 927279477: 'OPHYS_7_receptive_field_mapping',
 894836178: 'OPHYS_4_images_B',
 893993015: 'OPHYS_5_images_B_passive',
 893234928: 'OPHYS_6_images_B',
 888945661: 'OPHYS_3_images_A',
 886154222: 'OPHYS_2_images_A_passive',
 884353347: 'OPHYS_1_images_A',
 884545175: 'OPHYS_1_images_A',
 871524137: 'OPHYS_4_images_B',
 868615577: 'OPHYS_3_images_A',
 867016776: 'OPHYS_1_images_A',
 893940179: 'OPHYS_1_images_E',
 893238904: 'OPHYS_1_images_G',
 859459978: 'OPHYS_6_images_B',
 858899525: 'OPHYS_4_images_B',
 857759922: 'OPHYS_4_images_B',
 857075165: 'OPHYS_3_images_A',
 856337979: 'OPHYS_2_images_A_passive',
 855898523: 'OPHYS_7_receptive_field_mapping',
 852023722: 'OPHYS_7_receptive_field_mapping',
 847488953: 'OPHYS_1_images_A',
 841817171: 'OPHYS_7_receptiv